# Init ptbxl

In [1]:
import numpy as np
import wfdb
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels as stats
import os

In [7]:
path = '../data/physionet.org/files/ptb-xl/1.0.3/ptbxl_database.csv'
df = pd.read_csv(path)
df.head()


,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
1,2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
2,3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
3,4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
4,5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr


# Ziel Variable

In [9]:
df.to_csv('ptbxl_database.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21799 entries, 0 to 21798
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ecg_id                        21799 non-null  int64  
 1   patient_id                    21799 non-null  float64
 2   age                           21799 non-null  float64
 3   sex                           21799 non-null  int64  
 4   height                        6974 non-null   float64
 5   weight                        9421 non-null   float64
 6   nurse                         20326 non-null  float64
 7   site                          21782 non-null  float64
 8   device                        21799 non-null  object 
 9   recording_date                21799 non-null  object 
 10  report                        21799 non-null  object 
 11  scp_codes                     21799 non-null  object 
 12  heart_axis                    13331 non-null  object 
 13  i

In [17]:
print(f'Die Zielvariable hat {len(df["scp_codes"].unique())} unique values')

Die Zielvariable hat 5463 unique values


In [18]:
df['scp_codes'].head()

0    {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
1                {'NORM': 80.0, 'SBRAD': 0.0}
2                  {'NORM': 100.0, 'SR': 0.0}
3                  {'NORM': 100.0, 'SR': 0.0}
4                  {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, dtype: object

Die Zielvariable hat 5463 eindeutige Werte. Das liegt daran, dass sie in form von Dictionary ist (aber als String gespeichert ist), in dem viele verschiedene SCP-Codes enthalten sein können.

In [ ]:
mi = ["IMI", "ASMI", "ILMI", "AMI", "ALMI", "INJAS", "LMI", "INJAL", "IPLMI", "IPMI", "INJIN", "INJLA", "PMI", "INJIL"]

In [7]:
counter = 0
for value in df['scp_codes']:
    for element in mi:
        if element in value:
            counter += 1
            # Ein Patient darf nur ein mal als mi klassifiziert werden
            break

print(f'Die Anzahl der kranken Menschen: {counter}')
print(f'Die Anzahl der gesunden Menschen: {df.shape[0] - counter}')
print(f'Prozent der kranken Menschen: {counter/df.shape[0] * 100:#.{5}}%')

Die Anzahl der kranken Menschen: 5469
Die Anzahl der gesunden Menschen: 16330
Prozent der kranken Menschen: 25.088%


Hier wird die Zielvariable encoded

In [3]:
def target_encoder(x):
    """
    Falls eine der Arten von Herzinfarkt in SCP - Code von einer Person aufgelistet wurde, wird das als 1 klassifiziert, sonst 0
    """
    mi = ["IMI", "ASMI", "ILMI", "AMI", "ALMI", "INJAS", "LMI", "INJAL", "IPLMI", "IPMI", "INJIN", "INJLA", "PMI", "INJIL"]
    for element in mi:
        if element in x:
            return 1
    return 0


In [4]:
scp = df['scp_codes']
scp = scp.apply(target_encoder)

In [7]:
print(f'Die Anzahl der als 1 klassifizierten Personen: {scp[scp == 1].count().item()}')

Die Anzahl der als 1 klassifizierten Personen: 5469


In [11]:
cols = ['age', 'sex', 'height', 'weight', 'scp_codes', 'filename_lr', 'filename_hr']
df = df[cols]
df.head()

,age,sex,height,weight,scp_codes,filename_lr,filename_hr
0,56.0,1,NaN,63.0,0,records100/00000/00001_lr,records500/00000/00001_hr
1,19.0,0,NaN,70.0,0,records100/00000/00002_lr,records500/00000/00002_hr
2,37.0,1,NaN,69.0,0,records100/00000/00003_lr,records500/00000/00003_hr
3,24.0,0,NaN,82.0,0,records100/00000/00004_lr,records500/00000/00004_hr
4,19.0,1,NaN,70.0,0,records100/00000/00005_lr,records500/00000/00005_hr


# Fehlende Werte und Hypothesen Tests

In [12]:
df.isna().mean() * 100

age             0.000000
sex             0.000000
height         68.007707
weight         56.782421
scp_codes       0.000000
filename_lr     0.000000
filename_hr     0.000000
dtype: float64

es ist zu sehen dass beides Gewicht und Körpergrösse ziemlich viele fehlende Werte haben, es wird jetzt überprüft, ob diese systematisch fehlen oder zufällig

In [17]:
from feature_engine.imputation import AddMissingIndicator
def missing_vals_data_prep(df):
    cols = []
    for column in df.columns:
        cols.append(column)
    addBinary_imputer = AddMissingIndicator(
    variables=cols,
    )
    addBinary_imputer.fit(df)
    cols = []
    missings = addBinary_imputer.transform(df)
    return missings

missings = missing_vals_data_prep(df)

In [45]:
missings.head()

,age,sex,height,weight,scp_codes,filename_lr,filename_hr,height_na,weight_na
0,56.0,1,NaN,63.0,0,records100/00000/00001_lr,records500/00000/00001_hr,1,0
1,19.0,0,NaN,70.0,0,records100/00000/00002_lr,records500/00000/00002_hr,1,0
2,37.0,1,NaN,69.0,0,records100/00000/00003_lr,records500/00000/00003_hr,1,0
3,24.0,0,NaN,82.0,0,records100/00000/00004_lr,records500/00000/00004_hr,1,0
4,19.0,1,NaN,70.0,0,records100/00000/00005_lr,records500/00000/00005_hr,1,0


In [20]:
import scipy.stats as stats
def hypothesen_tests(missings):
    """
    Hypothesen Tests um die systematik der fehlenden Werten zu überprüfen
    """
    ps = {}
    sig_niv = 0.05
    for column in missings.columns:
        if column.endswith('_na'):
            na      = missings[missings[column] == 1]['scp_codes']
            nona    = missings[missings[column] == 0]['scp_codes']
            ps[column] = stats.ttest_ind(na,  nona, equal_var=False, alternative='two-sided').pvalue

    return ps
ps = hypothesen_tests(missings)
for element in ps:
    print(f'P-Wert von {element} = {ps[element]}')

P-Wert von height_na = 2.027644880075522e-32
P-Wert von weight_na = 1.6829221249770412e-159


P-Werte für beide Kategorien leigen unter dem signifikanz niveau alpha = 0.05 somit kann die Nullhypothese widerlegt werden. Die Werte sprechen dafür, dass die Werte systematisch fehlen.

In [29]:
print(f"Anteil der Menschen die Herzinfarkt hatten wenn die Körpergrösse angabe fehlt {missings[missings['height_na'] == 1]['scp_codes'].mean() * 100:#.{5}}%")
print(f"Anteil der Menschen die Herzinfarkt hatten wenn die Körpergrösse angabe nicht fehlt {missings[missings['height_na'] == 0]['scp_codes'].mean() * 100:#.{5}}%")

Anteil der Menschen die Herzinfarkt hatten wenn die Körpergrösse angabe fehlt 27.386%
Anteil der Menschen die Herzinfarkt hatten wenn die Körpergrösse angabe nicht fehlt 20.204%


In [30]:
print(f"Anteil der Menschen die Herzinfarkt hatten wenn die Gewichtangabe fehlt {missings[missings['weight_na'] == 1]['scp_codes'].mean() * 100:#.{5}}%")
print(f"Anteil der Menschen die Herzinfarkt hatten wenn die Gewichtangabe nicht fehlte {missings[missings['weight_na'] == 0]['scp_codes'].mean() * 100:#.{5}}%")

Anteil der Menschen die Herzinfarkt hatten wenn die Gewichtangabe fehlt 31.726%
Anteil der Menschen die Herzinfarkt hatten wenn die Gewichtangabe fehlt 16.368%


Noch ein test bezüglich der signifikanz von Geschlecht

In [33]:
na      = missings[missings['sex'] == 1]['scp_codes']
nona    = missings[missings['sex'] == 0]['scp_codes']
stats.ttest_ind(na,  nona, equal_var=False, alternative='two-sided').pvalue.item()

1.5076638905375842e-69

In [35]:
print(f"Anteil der Menschen mit Geschlecht 1 die Herzinfarkt hatten: {missings[missings['sex'] == 1]['scp_codes'].mean() * 100:#.{5}}%")
print(f"Anteil der Menschen mit Geschlecht 0 die Herzinfarkt hatten: {missings[missings['sex'] == 0]['scp_codes'].mean() * 100:#.{5}}%")

Anteil der Menschen mit Geschlecht 1 die Herzinfarkt hatten: 19.742%
Anteil der Menschen mit Geschlecht 0 die Herzinfarkt hatten: 30.007%


Es ist zu sehen, dass der p-wert ebenfalls unterhalb von 5% liegt, somit hat das Geschlecht einen signifikanten Einfluss auf die Zielvariable.

In [39]:
median = missings['age'].median().item()

In [44]:
young      = missings[missings['age'] < median]['scp_codes']
old    = missings[missings['age'] >= median]['scp_codes']
print(f'Die Anzahl der Menschen junger als Median: {young.count().item()}')
print(f'Die Anzahl der Menschen älter als Median: {old.count().item()}')
print(f"p-wert: {stats.ttest_ind(young, old, equal_var=False, alternative='two-sided').pvalue.item()}")
print(f"Anteil der jungen Menschen die Herzinfarkt hatten: {young.mean() * 100:#.{5}}%")
print(f"Anteil der alten Menschen die die Herzinfarkt hatten: {old.mean() * 100:#.{5}}%")

Die Anzahl der Menschen junger als Median: 10796
Die Anzahl der Menschen älter als Median: 11003
p-wert: 4.046075448422255e-153
Anteil der jungen Menschen die Herzinfarkt hatten: 17.349%
Anteil der alten Menschen die die Herzinfarkt hatten: 32.682%


Es ist ersichtlich, dass die Variable Alter auch signifikant für die Zielvariable ist.